In [0]:
'''Transfer learning toy example.

1 - Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
2 - Freeze convolutional layers and fine-tune dense layers
   for the classification of digits [5..9].

Get to 99.8% test accuracy after 5 epochs
for the first five digits classifier
and 99.2% for the last five digits after transfer + fine-tuning.
'''

from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


In [0]:
now = datetime.datetime.now

batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3


In [14]:
K.backend()

'tensorflow'

In [0]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)


In [0]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])


In [17]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5



In [19]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]


In [20]:
# create complete model
model = Sequential(feature_layers + classification_layers)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)


x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 30596 samples, validate on 5139 samples
Epoch 1/5





30596/30596 [==============================] - 51s 2ms/step - loss: 0.1832 - acc: 0.9411 - val_loss: 0.0320 - val_acc: 0.9897
Epoch 2/5
30596/30596 [==============================] - 50s 2ms/step - loss: 0.0504 - acc: 0.9850 - val_loss: 0.0150 - val_acc: 0.9946
Epoch 3/5
30596/30596 [==============================] - 50s 2ms/step - loss: 0.0343 - acc: 0.9892 - val_loss: 0.0091 - val_acc: 0.9975
Epoch 4/5
30596/30596 [==============================] - 50s 2ms/step - loss: 0.0261 - acc: 0.9919 - val_loss: 0.0087 - val_acc: 0.9971
Epoch 5/5
30596/30596 [==============================] - 50s 2ms/step - loss: 0.0210 - acc: 0.9937 - val_loss: 0.0069 - val_acc: 0.9977
Training time: 0:04:11.669548
Test score: 0.006855663326886841
Test accuracy: 0.997664915353181

In [0]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False


In [23]:

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 18s 600us/step - loss: 0.2469 - acc: 0.9289 - val_loss: 0.0528 - val_acc: 0.9844
Epoch 2/5
29404/29404 [==============================] - 17s 591us/step - loss: 0.0800 - acc: 0.9755 - val_loss: 0.0374 - val_acc: 0.9879
Epoch 3/5
29404/29404 [==============================] - 17s 592us/step - loss: 0.0580 - acc: 0.9822 - val_loss: 0.0309 - val_acc: 0.9897
Epoch 4/5
29404/29404 [==============================] - 17s 592us/step - loss: 0.0515 - acc: 0.9852 - val_loss: 0.0299 - val_acc: 0.9891
Epoch 5/5
29404/29404 [==============================] - 17s 592us/step - loss: 0.0439 - acc: 0.9864 - val_loss: 0.0265 - val_acc: 0.9907
Training time: 0:01:27.482087
Test score: 0.02652423154627661
Test accuracy: 0.9907426454358277
